# Figure 1 and Supp Fig 2: UMAPs of Nuclei Clustering at Leiden 0.85 Resolution
Visualization conceptualized by Raph Gibbs, Megan Duffy, and Monica Mesecar. 
AI coding support for aesthetics with Perplexity

In [ ]:
! date

## Install and Import Packages:

In [ ]:
#Install Pandas
!pip install pandas

#Install ScanPy
!pip install scanpy

In [ ]:
#Import ScanPy
import scanpy as sc

#Import Annotation data bject
import anndata as AnnData

#Import NumPy
import numpy as np

#Import where module
from numpy import where

#Import relevant Pandas functions
from pandas import read_csv, concat

#Import MatPlotLib/PyPlot
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context

#Import Seaborn
import seaborn as sns

#Import Warnings/Filter 
import warnings
warnings.simplefilter('ignore')

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

## Create Shorthand for Files/File Paths

In [ ]:
#Set up variables
# naming

# in files
full_file = '/data/ADRD/brain_aging/phase1/demux/aging.pegasus.leiden_085.subclustered.h5ad'

# out files
out_h5ad_file = '/data/ADRD/brain_aging/phase1/aging.pegasus.leiden_085.subclustered.full.annotated.h5ad'

# variables
DEBUG = True 
MIN_CELL_PERCENT = 0.005
MAX_MITO_PERCENT = 10
TESTING = False
testing_cell_size = 5000
FILTER_FEATURES_HV = True
TOP_FEATURES_PERCENT = 0.20

## Load Data

In [ ]:
%%time
#Read in-file to ScanPy
adata = sc.read(full_file)
print(adata)
if DEBUG:
    display(adata.obs.head())

In [ ]:
#When displaying UMAP, remove cells that are not definitively annotated or are mis-annotated
#Create mask for cells to remove
mask = ((adata.obs['broad_celltype'] == 'SPN') & (adata.obs['Brain_region'] != 'Putamen')) | (adata.obs['broad_celltype'] == 'Other') #Non-Putamen SPNs and "Other"

#Copy adata with mask removed so as not to overwrite 
adata = adata[~mask].copy()

## Create UMAP and Color by Leiden Cluster Label

In [ ]:
#Plot UMAP amd color and label by LEIDEN CLUSTER LABEL

#Figure creation, size, and resolution:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}): 
    #Specify color palette:
    plt.style.use('seaborn-v0_8-bright') #Must include -v_08- if using newer version. Remove these characters if not.
    #Specify data object source and what variable to color by:
    sc.pl.umap(adata, color=['leiden_labels_085'], 
               #Specify y/n frame and legend location:
               frameon=False, legend_loc='on data')

## Color UMAP by Brain Region of Origin

In [ ]:
#Specify Regional color palette dictionary
custom_palette_region = {
    'Entorhinal cortex': '#841F84',  
    'Middle temporal gyrus': '#2B79A9',  
    'Putamen': '#A8244B', 
    'Subventricular zone': '#E5B710',  
}
#Set Up Plot
with rc_context({'figure.figsize': (9, 9), 'figure.dpi': 300}):
    plt.style.use('seaborn-v0_8-bright') #Must include -v_08- if using newer version. Remove these characters if not.
    
    fig, ax = plt.subplots()
    
    sc.pl.umap(
        adata, #Data Object
        color=['Brain_region'], #Variable to color by
        palette=custom_palette_region,   #Custom regional color palette
        frameon=False,
        title='UMAP of Broad Cell Types by Brain Region\n(Leiden Resolution: 0.85)',
        ax=ax,
        show=False
    )
    #Add cell type labels as overlay
    for cluster in adata.obs['broad_celltype'].unique(): #Specify for a unique cell type
        #Add label overlay onto existing UMAP
        cells = adata[adata.obs['broad_celltype'] == cluster].obsm['X_umap']
        #Center labels on clusters 
        center = cells.mean(axis=0)
        ax.text(center[0], center[1], cluster, 
                fontweight='bold', ha='center', va='center')

    plt.tight_layout()
    plt.show()


## Color UMAP by Broad Cell Type Label

In [ ]:
#Originally Cell types were annotated more deeply but the decision was made to report at the level
#of broad cell type due to sampe size. So, cell subtypes with shared broad types were colored together 

custom_palette_cells = {
    'InN': '#3FA291',  # Teal Blue
    'InN ADARB2 LAMP5': '#3FA291',  # Teal Blue
    'InN ADARB2 VIP': '#3FA291',  # Teal Blue
    'InN LHX6 PVALB': '#3FA291',  # Teal Blue
    'InN LHX6 SST': '#3FA291',  # Teal Blue
    
    'ExN': '#D9C300', #Bright Yellow
    'ExN CUX2 ADARB2': '#D9C300', #Bright Yellow
    'ExN CUX2 LAMP5': '#D9C300', #Bright Yellow
    'ExN LAMP5': '#D9C300', #Bright Yellow
    'ExN FEZF2': '#D9C300', #Bright Yellow
    'ExN RORB': '#D9C300', #Bright Yellow
    'ExN RORB THEMIS': '#D9C300', #Bright Yellow
    'ExN THEMIS': '#D9C300', #Bright Yellow
    
    'SPN': '#514785', #Purple/Grey
    'SPN D1': '#514785', #Purple/Grey
    'SPN D1-2': '#514785', #Purple/Grey
    'SPN D2': '#514785', #Purple/Grey
    'SPN D2-2': '#514785', #Purple/Grey
    
    
    'Microglia': '#AD5E03',#Burnt Orange
    
    'Astrocyte': '#2B5B7D', #Blue/Grey
    'Astrocyte-GFAP-Hi': '#2B5B7D', #Blue/Grey
    
    'Oligodendrocyte': '#60861D', #Moss Green
    'Oligodendrocyte-1': '#60861D', #Moss Green
    'Oligodendrocyte-2': '#60861D', #Moss Green
    'Oligodendrocyte-3': '#60861D', #Moss Green
    
    'OPC': '#BB3A8A', #Fuschia 
    
    'Mural': '#B21706', #Deep red
    
    'Endothelial': '#683668', # Deep Violet
    
    'Ependymal': '#908005', #Green brown
   
    #Removed from dataset
    'Other': '#d9d9d9', #Light Grey
    'uncertain':  '#d9d9d9', #Light Grey
    'uncertain-2': '#d9d9d9', #Light Grey
    'uncertain-3': '#d9d9d9', #Light Grey

    'Radial Glia': '#d9d9d9', #Light Grey
}

# Set up the plot
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 300}):
    plt.style.use('seaborn-v0_8-bright') #Must include -v_08- if using newer version. Remove these characters if not.
    sc.pl.umap(adata, 
               color=['broad_celltype'], 
               frameon=False,
               legend_loc='on data',
               legend_fontsize=8,
               title='UMAP of Broad Cell Types\n(Leiden Resolution: 0.85)',
               palette=custom_palette_cells, #Custom cell type color palette
               save="Broad_CellType_UMAP_nolabel.png")
    plt.show()